In [36]:
from keras.models import load_model

import numpy as np
import random
import json
import sys

In [30]:
text = open('sayings.txt').read().lower()
text = text.replace('\xa0', ' ').replace('’', "'").replace('ﬁ', 'ft').replace('–', '_').replace('”', '"').replace('‘',"'")
print('corpus length:', len(text))

corpus length: 109940


In [31]:
d = {}
sofar = ''
key = None
for l in open('chars.js'):
    if ' = ' in l:
        if key:
            d[key] = json.loads(sofar)
        key, sofar = l.split(' = ')
    else:
        sofar += l
d[key] = json.loads(sofar)
d.keys()

dict_keys(['num_chars', 'indices_char', 'maxlen', 'char_indices'])

In [32]:
sofar

'{\n  "0": "\\n",\n  "1": " ",\n  "2": "!",\n  "3": "\\"",\n  "4": "\'",\n  "5": "(",\n  "6": ")",\n  "7": ",",\n  "8": "-",\n  "9": ".",\n  "10": "0",\n  "11": "1",\n  "12": "2",\n  "13": "3",\n  "14": "4",\n  "15": "5",\n  "16": "6",\n  "17": "7",\n  "18": "8",\n  "19": "9",\n  "20": ":",\n  "21": ";",\n  "22": "=",\n  "23": "?",\n  "24": "[",\n  "25": "]",\n  "26": "_",\n  "27": "a",\n  "28": "b",\n  "29": "c",\n  "30": "d",\n  "31": "e",\n  "32": "f",\n  "33": "g",\n  "34": "h",\n  "35": "i",\n  "36": "j",\n  "37": "k",\n  "38": "l",\n  "39": "m",\n  "40": "n",\n  "41": "o",\n  "42": "p",\n  "43": "q",\n  "44": "r",\n  "45": "s",\n  "46": "t",\n  "47": "u",\n  "48": "v",\n  "49": "w",\n  "50": "x",\n  "51": "y",\n  "52": "z",\n  "53": "\\u00e4",\n  "54": "\\u00e6",\n  "55": "\\u00e9",\n  "56": "\\u00eb"\n}\n'

In [41]:
char_indices = d['char_indices']
chars = ''.join(sorted(char_indices.keys()))
print('total chars:', len(chars))
indices_char = {int(k): v for k, v in d['indices_char'].items()}
maxlen = d['maxlen']
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

total chars: 57
nb sequences: 36634


In [34]:
errors = ''
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        if char in char_indices:
            X[i, t, char_indices[char]] = 1
        else:
            errors += char
        if next_chars[i] in char_indices:
            y[i, char_indices[next_chars[i]]] = 1
X.shape, y.shape, errors

((36634, 40, 57),
 (36634, 57),
 'ﬂﬂﬂﬂﬂﬂﬂﬂﬂﬂﬂﬂﬂﬂ//////////////////////////“““““““““““““———————————————————————————///////////////////////////——————————————/////////////——————————————−−−−−−−−−−−−−//////////////——————————————————————————/////////////………………………………………………………………………………………………………………………………………………““““““““““““““//////////////………………………………………………………………………………………………………………………………………………………………………………………………………………………&&&&&&&&&&&&&……………………………………//////////////……………………………………………………………………%%%%%%%%%%%%%“““““““““““““………………………………………………………………………………………………………………………………………………………………………………………………………………………//////////////……………………………………………………………………&&&&&&&&&&&&&â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99\x99â\x80â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99â\x80\x99\x80\x99')

In [38]:
model = load_model('nietzsche.h5')

In [39]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [45]:
for iteration in range(1, 4):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=256,
              epochs=5)

    start_index = random.randrange(len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/5
36634/36634 [==============================] - 18s 498us/step - loss: 1.2576 - acc: 0.6136
Epoch 2/5
36634/36634 [==============================] - 19s 525us/step - loss: 1.2139 - acc: 0.6268
Epoch 3/5
36634/36634 [==============================] - 20s 544us/step - loss: 1.1756 - acc: 0.6364
Epoch 4/5
36634/36634 [==============================] - 21s 577us/step - loss: 1.1449 - acc: 0.6447
Epoch 5/5
36634/36634 [==============================] - 20s 559us/step - loss: 1.1164 - acc: 0.6539

----- diversity: 0.2
----- Generating with seed: "ome to those who wait
all things must pa"
ome to those who wait
all things must pass.
the storm and lover.
the restle.
the same doctor and action when you want to the storm.
the same things are away to truth without and life.
be don't be an all in 

/Users/douwe/Dropbox/Douwe/Proj/notebooks/venv3/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


the same contempt.
learn like a storm the same cloud and things man war and best to stards of the storm.
the same childrer.
the strength.
let the rest's not the same problems and best the don't shame to the storm.
the same childrer or

----- diversity: 0.5
----- Generating with seed: "ome to those who wait
all things must pa"
ome to those who wait
all things must path is better than takes the beginning, the step.
how good and sable that is always richer of the reade the doin and live a problem of the other than the night things hear does not development of a schere and surrog.
the liad.
a borrow than it is the reading is anything of the little all time for the famed.
the you can't bothing to his morer things makes the like things belier.
stingour things alwa

----- diversity: 1.0
----- Generating with seed: "ome to those who wait
all things must pa"
ome to those who wait
all things must pain longing from have being particular confquerates out of the public man is and weat
the pain.
it 

In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12827226444526493741
]


In [46]:
model.save('sayings.h5')

In [10]:
import h5py

In [11]:
model.to_json()

'{"keras_version": "2.1.2", "class_name": "Sequential", "backend": "tensorflow", "config": [{"class_name": "LSTM", "config": {"unit_forget_bias": true, "batch_input_shape": [null, 40, 57], "return_state": false, "recurrent_activation": "hard_sigmoid", "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "seed": null, "distribution": "uniform"}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "implementation": 1, "dropout": 0.0, "units": 128, "recurrent_dropout": 0.0, "name": "lstm_1", "activation": "tanh", "go_backwards": false, "recurrent_constraint": null, "recurrent_initializer": {"class_name": "Orthogonal", "config": {"seed": null, "gain": 1.0}}, "trainable": true, "kernel_regularizer": null, "use_bias": true, "stateful": false, "dtype": "float32", "activity_regularizer": null, "bias_regularizer": null, "bias_constraint": null, "return_sequences": false, "recurrent_regularizer": null, "unroll": false, "kernel_constraint": n

In [16]:
with open('keras_js/chars.js', 'w') as fout:
    fout.write('maxlen = ' + str(maxlen) + '\n')
    fout.write('num_chars = ' + str(len(chars)) + '\n')
    fout.write('char_indices = ' + json.dumps(char_indices, indent=2) + '\n')
    fout.write('indices_char = ' + json.dumps(indices_char, indent=2) + '\n')


In [18]:
'\xa0'

'\xa0'

In [ ]:
'’' 